In [10]:
from datetime import datetime, timedelta
from pprint import pprint
import json
import requests

from src.params import *
from src.nws import NWSClient


In [16]:
client = NWSClient()
station = StationID.MDW
prior_days = 2

start = datetime.now(tz=BASE_TZ) - timedelta(days=prior_days)
end = datetime.now(tz=BASE_TZ)
# Try getting observation data
# res = client._request_observations(station, datetime.now(tz=BASE_TZ) - timedelta(days=prior_days), datetime.now(tz=BASE_TZ))
url = TIME_SERIES_BASE
params = {
    "radius": "kmdw,1",
    "start": start.astimezone(pytz.utc).strftime("%Y%m%d%H%M"),
    "end": end.astimezone(pytz.utc).strftime("%Y%m%d%H%M"),
    "vars": "air_temp,wind_speed,wind_direction,relative_humidity,air_temp_high_6_hour,air_temp_high_24_hour",
    "token": PUBLIC_TOKEN,
}
response = requests.get(url, params=params)
response.raise_for_status()
res = response.json()

print(res)
# json.dump(res, fp=(open('../tmp/observations.json', 'w')))

{'SUMMARY': {'NUMBER_OF_OBJECTS': 0, 'RESPONSE_CODE': -1, 'RESPONSE_MESSAGE': 'The station ID in RADIUS was not valid.', 'RESPONSE_TIME': 0, 'VERSION': 'v2.24.8'}}


In [9]:
from urlpath import URL
import requests
from pprint import pprint

# url = URL("https://api.weather.gov/products/1eae2f39-8187-42ad-ab00-1f212b40a93a")
url = URL("https://api.weather.gov/products/9c5a3929-9e59-46df-b406-a1aae6ca946c")
station = "FNT"
url = URL(f"https://forecast.weather.gov/product.php?site=DTX&issuedby={station}&product=CLI&format=CI&version=1&glossary=0")

response = requests.get(url)
response.raise_for_status()
print(response.text, file=open("../tmp/cli.html", 'w'))

In [12]:
lines = response.text.splitlines()
for i in range(len(lines) - 3):
    match_start = (f"CLI{station}" in lines[i] and "CLIMATE REPORT" in lines[i+2] and "NATIONAL WEATHER SERVICE" in lines[i+3])
    if match_start:
        match_start_i = i
        print(i)
    match_end = "$$" in lines[i]
    if match_end:
        match_end_i = i
        print(i)

print("\n".join(lines[match_start_i:match_end_i+1]))

461
559
CLIFNT

CLIMATE REPORT
NATIONAL WEATHER SERVICE DETROIT/PONTIAC MI
430 PM EST TUE NOV 12 2024

...................................

...THE FLINT MI CLIMATE SUMMARY FOR NOVEMBER 12 2024...
VALID TODAY AS OF 0400 PM LOCAL TIME.

CLIMATE NORMAL PERIOD 1991 TO 2020
CLIMATE RECORD PERIOD 1921 TO 2024


WEATHER ITEM   OBSERVED TIME   RECORD YEAR NORMAL DEPARTURE LAST
                VALUE   (LST)  VALUE       VALUE  FROM      YEAR
                                                  NORMAL
...................................................................
TEMPERATURE (F)
 TODAY
  MAXIMUM         50        MM  68    1927  48      2       51
                                      1964
                                      2010
  MINIMUM         35        MM   6    2019  32      3       30
  AVERAGE         43                        40      3       41

PRECIPITATION (IN)
  TODAY            0.00          1.35 1992   0.08  -0.08     0.00
  MONTH TO DATE    0.54                      0.96  -0.

In [ ]:
from src.nws import NWSClient
from src.params import *
from datetime import datetime, timedelta

client = NWSClient()
res = client.get_timeseries(StationID.AUS, start=datetime.now() - timedelta(days=2), end=datetime.now())

res

In [ ]:
res["STATION"][0]["OBSERVATIONS"].keys()
res["STATION"][0]["OBSERVATIONS"]["air_temp_set_1"]

In [ ]:
from src.nws import NWSClient
from src.params import *

client = NWSClient()
client.get_all_cli_data(StationID.AUS)

In [ ]:
client.get_all_cli_data(StationID.AUS)


In [ ]:
from dataclasses import dataclass
from datetime import datetime, date
import re
from typing import Optional
from pprint import pprint

@dataclass
class ParsedCLIReport:
    issuance_time: datetime
    summary_date: date
    product_code: str
    product_name: str
    issuing_office: str
    max_temp: Optional[int] = None
    max_temp_time: Optional[datetime] = None
    min_temp: Optional[int] = None
    min_temp_time: Optional[datetime] = None
    avg_temp: Optional[int] = None
    precipitation: Optional[float] = None
    precipitation_month_to_date: Optional[float] = None
    precipitation_since_sep_1: Optional[float] = None
    precipitation_since_jan_1: Optional[float] = None
    highest_wind_speed: Optional[int] = None
    highest_wind_direction: Optional[str] = None
    highest_gust_speed: Optional[int] = None
    highest_gust_direction: Optional[str] = None
    average_wind_speed: Optional[float] = None
    average_sky_cover: Optional[float] = None
    highest_humidity: Optional[int] = None
    lowest_humidity: Optional[int] = None
    average_humidity: Optional[int] = None
    

def _parse_product_text(product_text: str, issuance_time: datetime, product_code: str, product_name: str, issuing_office: str) -> ParsedCLIReport:
    # Extract relevant sections using regular expressions

    date_match = re.search(r"THE AUSTIN BERGSTROM CLIMATE SUMMARY FOR (.+?)\.\.\.", product_text)

    temp_pattern = re.compile(r"""
        (MAXIMUM|MINIMUM|AVERAGE)\s+         # Match the label (MAXIMUM, MINIMUM, AVERAGE)
        (\d+)\s+                             # Match the observed value
        (\d{1,2}:\d{2}\s[APM]{2})?\s*        # Match the time (optional for AVERAGE)
        (\d+)?\s*                            # Match the record value (optional for AVERAGE)
        (\d{4})?\s*                          # Match the record year (optional for AVERAGE)
        (\d+)?\s*                            # Match the normal value (optional for AVERAGE)
        ([+-]?\d+)?\s*                       # Match the departure from normal (optional for AVERAGE)
        (\d+)?                               # Match the last year value (optional for AVERAGE)
    """, re.VERBOSE)

    matches = temp_pattern.findall(product_text)

    temp_data = {
        "max_temp": None,
        "max_temp_time": None,
        "min_temp": None,
        "min_temp_time": None,
        "avg_temp": None,
    }
    

    for match in matches:
        label, observed_value, time, *_ = match
        if label == "MAXIMUM":
            temp_data["max_temp"] = int(observed_value)
            if time:
                temp_data["max_temp_time"] = datetime.strptime(f"{date_match.group(1)} {time}", "%B %d %Y %I:%M %p")
        elif label == "MINIMUM":
            temp_data["min_temp"] = int(observed_value)
            if time:
                temp_data["min_temp_time"] = datetime.strptime(f"{date_match.group(1)} {time}", "%B %d %Y %I:%M %p")
        elif label == "AVERAGE":
            temp_data["avg_temp"] = int(observed_value)

    # precip_pattern = re.compile(r"""
    #     PRECIPITATION\s+\(IN\)\s+            # Match the PRECIPITATION header
    #     (?:.*?\n)+?                          # Non-greedy match to skip lines until labels
    #     (YESTERDAY|TODAY|MONTH TO DATE|SINCE SEP 1|SINCE JAN 1)\s+  # Match the label
    #     (\d+\.\d+)\s+                                         # Match the observed value
    #     (\d+\.\d+)?\s*                                        # Match the record value (optional)
    #     (\d{4})?\s*                                           # Match the record year (optional)
    #     (\d+\.\d+)?\s*                                        # Match the normal value (optional)
    #     ([+-]?\d+\.\d+)?\s*                                   # Match the departure from normal (optional)
    #     (\d+\.\d+)?                                           # Match the last year value (optional)
    # """, re.VERBOSE)

    # matches = precip_pattern.findall(product_text)

    # precipitation_data = {
    #     "day": None,
    #     "month_to_date": None,
    #     "since_sep_1": None,
    #     "since_jan_1": None
    # }

    # for match in matches:
    #     label, observed_value, *_ = match
    #     print(label, observed_value)
    #     if label == "YESTERDAY" or label == "TODAY":
    #         precipitation_data["day"] = float(observed_value)
    #     elif label == "MONTH TO DATE":
    #         precipitation_data["month_to_date"] = float(observed_value)
    #     elif label == "SINCE SEP 1":
    #         precipitation_data["since_sep_1"] = float(observed_value)
    #     elif label == "SINCE JAN 1":
    #         precipitation_data["since_jan_1"] = float(observed_value)

    highest_wind_speed_match = re.search(r"HIGHEST WIND SPEED\s+(\d+)", product_text)
    highest_wind_direction_match = re.search(r"HIGHEST WIND DIRECTION\s+(\w+)", product_text)
    highest_gust_speed_match = re.search(r"HIGHEST GUST SPEED\s+(\d+)", product_text)
    highest_gust_direction_match = re.search(r"HIGHEST GUST DIRECTION\s+(\w+)", product_text)
    average_wind_speed_match = re.search(r"AVERAGE WIND SPEED\s+(\d+\.\d+)", product_text)
    average_sky_cover_match = re.search(r"AVERAGE SKY COVER\s+(\d+\.\d+)", product_text)
    highest_humidity_match = re.search(r"HIGHEST\s+(\d+)", product_text)
    lowest_humidity_match = re.search(r"LOWEST\s+(\d+)", product_text)
    average_humidity_match = re.search(r"AVERAGE\s+(\d+)", product_text)

    # Convert date and time strings to date and datetime objects
    summary_date_obj = datetime.strptime(date_match.group(1), "%B %d %Y").date() if date_match else None

    return ParsedCLIReport(
        issuance_time=issuance_time,
        summary_date=summary_date_obj,
        max_temp=temp_data["max_temp"],
        max_temp_time=temp_data["max_temp_time"],
        min_temp=temp_data["min_temp"],
        min_temp_time=temp_data["min_temp_time"],
        avg_temp=temp_data["avg_temp"],
        # precipitation=precipitation_data["day"],
        # precipitation_month_to_date=precipitation_data["month_to_date"],
        # precipitation_since_sep_1=precipitation_data["since_sep_1"],
        # precipitation_since_jan_1=precipitation_data["since_jan_1"],
        highest_wind_speed=int(highest_wind_speed_match.group(1)) if highest_wind_speed_match else None,
        highest_wind_direction=highest_wind_direction_match.group(1) if highest_wind_direction_match else None,
        highest_gust_speed=int(highest_gust_speed_match.group(1)) if highest_gust_speed_match else None,
        highest_gust_direction=highest_gust_direction_match.group(1) if highest_gust_direction_match else None,
        average_wind_speed=float(average_wind_speed_match.group(1)) if average_wind_speed_match else None,
        average_sky_cover=float(average_sky_cover_match.group(1)) if average_sky_cover_match else None,
        highest_humidity=int(highest_humidity_match.group(1)) if highest_humidity_match else None,
        lowest_humidity=int(lowest_humidity_match.group(1)) if lowest_humidity_match else None,
        average_humidity=int(average_humidity_match.group(1)) if average_humidity_match else None,
        product_code=product_code,
        product_name=product_name,
        issuing_office=issuing_office
    )

def parse_json(json_data: dict) -> ParsedCLIReport:
    issuance_time = datetime.fromisoformat(json_data['issuanceTime'])
    product_code = json_data['productCode']
    product_name = json_data['productName']
    issuing_office = json_data['issuingOffice']
    product_text = json_data['productText']

    # print(product_text)
    parsed_report = _parse_product_text(product_text, issuance_time, product_code, product_name, issuing_office)

    return parsed_report

# Example usage
import requests

url = "https://api.weather.gov/products/1eae2f39-8187-42ad-ab00-1f212b40a93a"
response = requests.get(url)
response.raise_for_status()
data = response.json()

parsed_report = parse_json(data)
pprint(parsed_report)

In [ ]:
from datetime import datetime

product_text = """000
CDUS44 KEWX 112332
CLIAUS

CLIMATE REPORT 
NATIONAL WEATHER SERVICE AUSTIN/SAN ANTONIO
532 PM CST MON NOV 11 2024

...................................

...THE AUSTIN BERGSTROM CLIMATE SUMMARY FOR NOVEMBER 11 2024...
VALID TODAY AS OF 0500 PM LOCAL TIME.

CLIMATE NORMAL PERIOD: 1991 TO 2020
CLIMATE RECORD PERIOD: 1943 TO 2024


WEATHER ITEM   OBSERVED TIME   RECORD YEAR NORMAL DEPARTURE LAST      
                VALUE   (LST)  VALUE       VALUE  FROM      YEAR     
                                                  NORMAL           
...................................................................
TEMPERATURE (F)                                                          
 TODAY                                                               
  MAXIMUM         85   3:49 PM  87    2015  73     12       62        
  MINIMUM         54   7:01 AM  26    1950  48      6       54        
                                      2011                           
  AVERAGE         70                        61      9       58     

PRECIPITATION (IN)                                                    
  TODAY            0.00          1.91 1957   0.10  -0.10      T       
  MONTH TO DATE    1.37                      1.21   0.16     0.76     
  SINCE SEP 1      1.45                      8.49  -7.04     8.09     
  SINCE JAN 1     28.09                     31.49  -3.40    24.07     

SNOWFALL (IN)                                                              
  TODAY            0.0                                                
  MONTH TO DATE    0.0                                                
  SINCE SEP 1      0.0                                                
  SINCE JUL 1      0.0                                                
  SNOW DEPTH       0                                                 

DEGREE DAYS                                                           
 HEATING                                                              
  TODAY            0                         6     -6        7        
  MONTH TO DATE    6                        56    -50       52        
  SINCE SEP 1     13                       101    -88      104        
  SINCE JUL 1     13                       101    -88      104        

 COOLING                                                              
  TODAY            5                         2      3        0        
  MONTH TO DATE   89                        26     63       31        
  SINCE SEP 1    934                       644    290      903        
  SINCE JAN 1   3486                      2960    526     3477        
...................................................................


WIND (MPH)                                                            
  HIGHEST WIND SPEED     9   HIGHEST WIND DIRECTION    NE (50)        
  HIGHEST GUST SPEED    16   HIGHEST GUST DIRECTION     E (70)        
  AVERAGE WIND SPEED     2.9                                        


SKY COVER                                                             
  AVERAGE SKY COVER 0.1                                                 


WEATHER CONDITIONS                                                    
THE FOLLOWING WEATHER WAS RECORDED TODAY.                             
  NO SIGNIFICANT WEATHER WAS OBSERVED.                                  


RELATIVE HUMIDITY (PERCENT)
 HIGHEST    97           5:00 AM                                      
 LOWEST     41           5:00 PM                                      
 AVERAGE    69                                                        

..........................................................


THE AUSTIN BERGSTROM CLIMATE NORMALS FOR TOMORROW
                         NORMAL    RECORD    YEAR                     
 MAXIMUM TEMPERATURE (F)   73        87      1988                      
 MINIMUM TEMPERATURE (F)   48        26      1950                      


SUNRISE AND SUNSET                                                    
NOVEMBER 11 2024......SUNRISE   6:53 AM CST   SUNSET   5:36 PM CST     
NOVEMBER 12 2024......SUNRISE   6:54 AM CST   SUNSET   5:35 PM CST     


-  INDICATES NEGATIVE NUMBERS.
R  INDICATES RECORD WAS SET OR TIED.
MM INDICATES DATA IS MISSING.
T  INDICATES TRACE AMOUNT.




$$"""

lines = [l.strip() for l in product_text.splitlines()]

for line in lines:
    

issuance_time = datetime.strptime(lines[6], '%I%M %p %Z %a %b %d %Y')

summary_date_str = " ".join(lines[10].strip("...").split()[-3:])
summary_time_str = " ".join(lines[11].split()[-4:-2])
summary_datetime = datetime.strptime(f"{summary_date_str} {summary_time_str}", '%B %d %Y %I%M %p')

temp_max_details = lines[23].split()
temp_max = int(temp_max_details[1])
temp_max_time = datetime.strptime(" ".join(temp_max_details[2:4]), '%I:%M %p').time()
temp_min_details = lines[24].split()
temp_min = int(temp_min_details[1])
temp_min_time = datetime.strptime(" ".join(temp_min_details[2:4]), '%I:%M %p').time()

precip_day = float(lines[29].split()[1])
precip_month = float(lines[30].split()[1])
precip_ = float(lines[31].split()[1])

# print(" ".join(lines[10].strip("...").split(' ')[-3:]))
# summary_date = datetime.strptime(lines[10].split(' ')[-1], '%b %d %Y').date()

issuance_time, summary_datetime, temp_max, temp_max_time, temp_min, temp_min_time

In [ ]:
from dataclasses import dataclass
from datetime import datetime, date
from typing import Optional

@dataclass
class ParsedCLIReport:
    issuance_time: datetime
    summary_date: date
    product_code: str
    product_name: str
    issuing_office: str
    max_temp: Optional[int] = None
    max_temp_time: Optional[datetime] = None
    min_temp: Optional[int] = None
    min_temp_time: Optional[datetime] = None
    avg_temp: Optional[int] = None
    precipitation: Optional[float] = None
    precipitation_month_to_date: Optional[float] = None
    precipitation_since_sep_1: Optional[float] = None
    precipitation_since_jan_1: Optional[float] = None
    highest_wind_speed: Optional[int] = None
    highest_wind_direction: Optional[str] = None
    highest_gust_speed: Optional[int] = None
    highest_gust_direction: Optional[str] = None
    average_wind_speed: Optional[float] = None
    average_sky_cover: Optional[float] = None
    highest_humidity: Optional[int] = None
    lowest_humidity: Optional[int] = None
    average_humidity: Optional[int] = None

    ['000',
 'CDUS44 KEWX 112332',
 'CLIAUS',
 '',
 'CLIMATE REPORT ',
 'NATIONAL WEATHER SERVICE AUSTIN/SAN ANTONIO',
 '532 PM CST MON NOV 11 2024',
 '',
 '...................................',
 '',
 '...THE AUSTIN BERGSTROM CLIMATE SUMMARY FOR NOVEMBER 11 2024...',
 'VALID TODAY AS OF 0500 PM LOCAL TIME.',
 '',
 'CLIMATE NORMAL PERIOD: 1991 TO 2020',
 'CLIMATE RECORD PERIOD: 1943 TO 2024',
 '',
 '',
 'WEATHER ITEM   OBSERVED TIME   RECORD YEAR NORMAL DEPARTURE LAST      ',
 '                VALUE   (LST)  VALUE       VALUE  FROM      YEAR     ',
 '                                                  NORMAL           ',
 '...................................................................',
 'TEMPERATURE (F)                                                          ',
 ' TODAY                                                               ',
 '  MAXIMUM         85   3:49 PM  87    2015  73     12       62        ',
 '  MINIMUM         54   7:01 AM  26    1950  48      6       54        ',
 '                                      2011                           ',
 '  AVERAGE         70                        61      9       58     ',
 '',
 'PRECIPITATION (IN)                                                    ',
 '  TODAY            0.00          1.91 1957   0.10  -0.10      T       ',
 '  MONTH TO DATE    1.37                      1.21   0.16     0.76     ',
 '  SINCE SEP 1      1.45                      8.49  -7.04     8.09     ',
 '  SINCE JAN 1     28.09                     31.49  -3.40    24.07     ',
 '',
 'SNOWFALL (IN)                                                              ',
 '  TODAY            0.0                                                ',
 '  MONTH TO DATE    0.0                                                ',
 '  SINCE SEP 1      0.0                                                ',
 '  SINCE JUL 1      0.0                                                ',
 '  SNOW DEPTH       0                                                 ',
 '',
 'DEGREE DAYS                                                           ',
 ' HEATING                                                              ',
 '  TODAY            0                         6     -6        7        ',
 '  MONTH TO DATE    6                        56    -50       52        ',
 '  SINCE SEP 1     13                       101    -88      104        ',
 '  SINCE JUL 1     13                       101    -88      104        ',
 '',
 ' COOLING                                                              ',
 '  TODAY            5                         2      3        0        ',
 '  MONTH TO DATE   89                        26     63       31        ',
 '  SINCE SEP 1    934                       644    290      903        ',
 '  SINCE JAN 1   3486                      2960    526     3477        ',
 '...................................................................',
 '',
 '',
 'WIND (MPH)                                                            ',
 '  HIGHEST WIND SPEED     9   HIGHEST WIND DIRECTION    NE (50)        ',
 '  HIGHEST GUST SPEED    16   HIGHEST GUST DIRECTION     E (70)        ',
 '  AVERAGE WIND SPEED     2.9                                        ',
 '',
 '',
 'SKY COVER                                                             ',
 '  AVERAGE SKY COVER 0.1                                                 ',
 '',
 '',
 'WEATHER CONDITIONS                                                    ',
 'THE FOLLOWING WEATHER WAS RECORDED TODAY.                             ',
 '  NO SIGNIFICANT WEATHER WAS OBSERVED.                                  ',
 '',
 '',
 'RELATIVE HUMIDITY (PERCENT)',
 ' HIGHEST    97           5:00 AM                                      ',
 ' LOWEST     41           5:00 PM                                      ',
 ' AVERAGE    69                                                        ',
 '',
 '..........................................................',
 '',
 '',
 'THE AUSTIN BERGSTROM CLIMATE NORMALS FOR TOMORROW',
 '                         NORMAL    RECORD    YEAR                     ',
 ' MAXIMUM TEMPERATURE (F)   73        87      1988                      ',
 ' MINIMUM TEMPERATURE (F)   48        26      1950                      ',
 '',
 '',
 'SUNRISE AND SUNSET                                                    ',
 'NOVEMBER 11 2024......SUNRISE   6:53 AM CST   SUNSET   5:36 PM CST     ',
 'NOVEMBER 12 2024......SUNRISE   6:54 AM CST   SUNSET   5:35 PM CST     ',
 '',
 '',
 '-  INDICATES NEGATIVE NUMBERS.',
 'R  INDICATES RECORD WAS SET OR TIED.',
 'MM INDICATES DATA IS MISSING.',
 'T  INDICATES TRACE AMOUNT.',
 '',
 '',
 '',
 '',
 '$$']


def parse_product_text(product_text: str) -> ParsedCLIReport:
    lines = product_text.splitlines()
    pprint(lines)
    
    issuance_time = datetime.strptime(lines[6], '%I%M %p %Z %a %b %d %Y')
    summary_date = datetime.strptime(lines[10].split(' ')[-1], '%b %d %Y').date()
    product_code = lines[1].split(' ')[1]
    product_name = lines[3].strip()
    issuing_office = lines[5].strip()
    
    max_temp = int(lines[17].split()[1])
    max_temp_time = datetime.strptime(lines[17].split()[2] + ' ' + lines[17].split()[3], '%I:%M %p')
    min_temp = int(lines[18].split()[1])
    min_temp_time = datetime.strptime(lines[18].split()[2] + ' ' + lines[18].split()[3], '%I:%M %p')
    avg_temp = int(lines[19].split()[1])
    
    precipitation = float(lines[22].split()[1])
    precipitation_month_to_date = float(lines[23].split()[2])
    precipitation_since_sep_1 = float(lines[24].split()[2])
    precipitation_since_jan_1 = float(lines[25].split()[2])
    
    highest_wind_speed = int(lines[30].split()[3])
    highest_wind_direction = lines[30].split()[6]
    highest_gust_speed = int(lines[31].split()[3])
    highest_gust_direction = lines[31].split()[6]
    average_wind_speed = float(lines[32].split()[3])
    
    average_sky_cover = float(lines[35].split()[3])
    
    highest_humidity = int(lines[41].split()[1])
    lowest_humidity = int(lines[42].split()[1])
    average_humidity = int(lines[43].split()[1])
    
    return ParsedCLIReport(
        issuance_time=issuance_time,
        summary_date=summary_date,
        product_code=product_code,
        product_name=product_name,
        issuing_office=issuing_office,
        max_temp=max_temp,
        max_temp_time=max_temp_time,
        min_temp=min_temp,
        min_temp_time=min_temp_time,
        avg_temp=avg_temp,
        precipitation=precipitation,
        precipitation_month_to_date=precipitation_month_to_date,
        precipitation_since_sep_1=precipitation_since_sep_1,
        precipitation_since_jan_1=precipitation_since_jan_1,
        highest_wind_speed=highest_wind_speed,
        highest_wind_direction=highest_wind_direction,
        highest_gust_speed=highest_gust_speed,
        highest_gust_direction=highest_gust_direction,
        average_wind_speed=average_wind_speed,
        average_sky_cover=average_sky_cover,
        highest_humidity=highest_humidity,
        lowest_humidity=lowest_humidity,
        average_humidity=average_humidity
    )

product_text = """000
CDUS44 KEWX 112332
CLIAUS

CLIMATE REPORT 
NATIONAL WEATHER SERVICE AUSTIN/SAN ANTONIO
532 PM CST MON NOV 11 2024

...................................

...THE AUSTIN BERGSTROM CLIMATE SUMMARY FOR NOVEMBER 11 2024...
VALID TODAY AS OF 0500 PM LOCAL TIME.

CLIMATE NORMAL PERIOD: 1991 TO 2020
CLIMATE RECORD PERIOD: 1943 TO 2024


WEATHER ITEM   OBSERVED TIME   RECORD YEAR NORMAL DEPARTURE LAST      
                VALUE   (LST)  VALUE       VALUE  FROM      YEAR     
                                                  NORMAL           
...................................................................
TEMPERATURE (F)                                                          
 TODAY                                                               
  MAXIMUM         85   3:49 PM  87    2015  73     12       62        
  MINIMUM         54   7:01 AM  26    1950  48      6       54        
                                      2011                           
  AVERAGE         70                        61      9       58     

PRECIPITATION (IN)                                                    
  TODAY            0.00          1.91 1957   0.10  -0.10      T       
  MONTH TO DATE    1.37                      1.21   0.16     0.76     
  SINCE SEP 1      1.45                      8.49  -7.04     8.09     
  SINCE JAN 1     28.09                     31.49  -3.40    24.07     

SNOWFALL (IN)                                                              
  TODAY            0.0                                                
  MONTH TO DATE    0.0                                                
  SINCE SEP 1      0.0                                                
  SINCE JUL 1      0.0                                                
  SNOW DEPTH       0                                                 

DEGREE DAYS                                                           
 HEATING                                                              
  TODAY            0                         6     -6        7        
  MONTH TO DATE    6                        56    -50       52        
  SINCE SEP 1     13                       101    -88      104        
  SINCE JUL 1     13                       101    -88      104        

 COOLING                                                              
  TODAY            5                         2      3        0        
  MONTH TO DATE   89                        26     63       31        
  SINCE SEP 1    934                       644    290      903        
  SINCE JAN 1   3486                      2960    526     3477        
...................................................................


WIND (MPH)                                                            
  HIGHEST WIND SPEED     9   HIGHEST WIND DIRECTION    NE (50)        
  HIGHEST GUST SPEED    16   HIGHEST GUST DIRECTION     E (70)        
  AVERAGE WIND SPEED     2.9                                        


SKY COVER                                                             
  AVERAGE SKY COVER 0.1                                                 


WEATHER CONDITIONS                                                    
THE FOLLOWING WEATHER WAS RECORDED TODAY.                             
  NO SIGNIFICANT WEATHER WAS OBSERVED.                                  


RELATIVE HUMIDITY (PERCENT)
 HIGHEST    97           5:00 AM                                      
 LOWEST     41           5:00 PM                                      
 AVERAGE    69                                                        

..........................................................


THE AUSTIN BERGSTROM CLIMATE NORMALS FOR TOMORROW
                         NORMAL    RECORD    YEAR                     
 MAXIMUM TEMPERATURE (F)   73        87      1988                      
 MINIMUM TEMPERATURE (F)   48        26      1950                      


SUNRISE AND SUNSET                                                    
NOVEMBER 11 2024......SUNRISE   6:53 AM CST   SUNSET   5:36 PM CST     
NOVEMBER 12 2024......SUNRISE   6:54 AM CST   SUNSET   5:35 PM CST     


-  INDICATES NEGATIVE NUMBERS.
R  INDICATES RECORD WAS SET OR TIED.
MM INDICATES DATA IS MISSING.
T  INDICATES TRACE AMOUNT.




$$"""

parsed_report = parse_product_text(product_text)
print(parsed_report)

In [ ]:
from cfgrib import dataset, messages
import os
import xarray as xr

from src.params import *

data_fp = INPUTS_DIR / "nam_218_20241006_0000_002.grb2"

def open_file(
    path,
    errors: str = "warn",
    indexpath: str = messages.DEFAULT_INDEXPATH,
    filter_by_keys = {},
    read_keys= (),
    time_dims= ("time", "step"),
    extra_coords= {},
    ignore_keys = [],
    **kwargs,
):
    """Open a GRIB file as a ``cfgrib.Dataset``."""
    path = os.fspath(path)
    stream = messages.FileStream(path, errors=errors)
    index_keys = dataset.compute_index_keys(time_dims, extra_coords)
    index = dataset.open_fileindex(stream, indexpath, index_keys, ignore_keys=ignore_keys, filter_by_keys=filter_by_keys)
    return index


def enforce_unique_attributes(index, attributes_keys=dataset.DATA_ATTRIBUTES_KEYS, filter_by_keys={}):
    attributes = {}
    for key in attributes_keys:
        values = index.get(key, [])
        print(key)
        print(values)
        # if len(values) > 1:
        #     fbks = []
        #     for value in values:
        #         fbk = {key: value}
        #         fbk.update(filter_by_keys)
        #         fbks.append(fbk)

index = open_file(data_fp)
enforce_unique_attributes(index, filter_by_keys={"typeOfLevel": "surface", "stepType": "instant"})

In [ ]:
import xarray as xr

from src.params import *

# filter_by_keys={'typeOfLevel': 'meanSea'}
# filter_by_keys={'typeOfLevel': 'hybrid'}
# filter_by_keys={'typeOfLevel': 'atmosphereSingleLayer'}
# filter_by_keys={'typeOfLevel': 'surface'}
# filter_by_keys={'typeOfLevel': 'cloudTop'}
# filter_by_keys={'typeOfLevel': 'planetaryBoundaryLayer'}
# filter_by_keys={'typeOfLevel': 'isobaricInhPa'}
# filter_by_keys={'typeOfLevel': 'heightAboveGround'}
# filter_by_keys={'typeOfLevel': 'depthBelowLandLayer'}
# filter_by_keys={'typeOfLevel': 'depthBelowLand'}
# filter_by_keys={'typeOfLevel': 'lowestLevelWetBulb0'}
# filter_by_keys={'typeOfLevel': 'isobaricLayer'}
# filter_by_keys={'typeOfLevel': 'cloudBase'}
# filter_by_keys={'typeOfLevel': 'heightAboveGroundLayer'}
# filter_by_keys={'typeOfLevel': 'tropopause'}
# filter_by_keys={'typeOfLevel': 'maxWind'}
# filter_by_keys={'typeOfLevel': 'isothermZero'}
# filter_by_keys={'typeOfLevel': 'pressureFromGroundLayer'}
# filter_by_keys={'typeOfLevel': 'sigmaLayer'}

data_fp = INPUTS_DIR / "nam_218_20241006_0000_002.grb2"
ds = xr.open_dataset(data_fp, engine="cfgrib", filter_by_keys={"typeOfLevel": "surface"})

In [ ]:
print(ds)

In [ ]:
from timeit import timeit

from src.params import *

from src.nws import NWSClient

first_time = timeit(lambda: NWSClient().get_location_info(NYC[0], NYC[1]), number=2)
print(f"First time: {first_time}")

client = NWSClient()
client.get_location_info(NYC[0], NYC[1])

second_time = timeit(lambda: client.get_location_info(NYC[0], NYC[1]), number=2)
print(f"Second time: {second_time}")

In [1]:
from pprint import pprint
import requests

from src.params import *
from src.nws import NWSClient
# intForecastFeatureFlags

client = NWSClient()
url = client.get_forecast_url(*NYC)
res1 = requests.get(url).json()
url = client.get_hourly_forecast_url(*NYC)
res2 = requests.get(url).json()

# max(res['properties']['periods']['temperature']['values'], key=lambda x: x['validTime'])
# client.get_location_info(CENTRAL_PARK[0], CENTRAL_PARK[1])
forecast = client.get_forecast_data(*NYC)
hourly_forecast = client.get_hourly_forecast_data(*NYC)
# pprint(client.get_hourly_forecast_data(*CENTRAL_PARK))

# print(forecast)
# pprint(res)

In [ ]:
res1

In [ ]:
pprint(forecast)

In [ ]:
pprint(client.get_hourly_forecast_data(*NYC))

In [ ]:
from urlpath import URL
URL("https://api.weather.gov") / "points" / "39.7456,-97.0892" / "forecast"

In [ ]:
lst = ["ca",
"co",
"ct",
"de",
"hi",
"il",
"md",
"ma",
"nj",
"nm",
"ny",
"or",
"ri",
"vt",
"va",
"wa",]

lst = [x.upper() for x in lst]
print(lst)